In [1]:
# pip install boto3

In [2]:
# pip install awscli

In [2]:
import gradio as gr
from transformers import pipeline
from dotenv import load_dotenv
import os
import subprocess
import torch
#Google Text to Speech
from gtts import gTTS
import tempfile

import boto3

In [10]:
#Create a Polly client
polly_client = boto3.client('polly')

# Text to speech
response = polly_client.synthesize_speech(
    Engine= 'neural',
    Text='Hello, how are you? My name is Gregory and I need to say more words to find out how I sound',
    OutputFormat='mp3',
    VoiceId='Gregory'
)

# Save the audio to a file
with open('speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())

print("Audio saved as speech.mp3")

Audio saved as speech.mp3


In [11]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
# This function transcribes audio to text using Whisper
def transcribe_audio(audio_filepath):
    try:
        transcription_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-large")
        transcription_result = transcription_pipeline(audio_filepath)
        transcribed_text = transcription_result['text']
        return transcribed_text
    except Exception as e:
        print(f"an error occured: {e}")
        return "Error in transcription"

In [13]:
# Define function to translate text to speech for output
# Uses Google Text-to-speech

def text_to_speech(text):
    tts = gTTS(text, lang='en')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
    tts.save(temp_file.name)
    return temp_file.name

In [17]:
# create combined function with both transcribe audio and text_to_speech

def combined_function (audio_filepath):
    transcribed_text = transcribe_audio(audio_filepath)
    speech_file_path = text_to_speech(transcribed_text)
    return transcribed_text, speech_file_path


In [18]:
# Create Gradio app to:
# 1. transcribe spoken audio to text
# 2. output transcribed text as speech

input_audio = gr.Audio(
            label="click on microphone to record audio", 
            type="filepath", 
            #vaveform options customize the color of the wave seen when recording/playing.
            waveform_options = gr.WaveformOptions(
                waveform_color="#01C6FF",
                waveform_progress_color="#0066B4",
                skip_length=2,
                show_controls=False,
            ),
)

app = gr.Interface(
    fn=combined_function,
    inputs=input_audio,
    outputs=[
        gr.Textbox(label="Transcribed audio"),
        gr.Audio(label="Text to speech output")],
    title="Audio Transcription and Text to Speech",
    description="click on the microphone to record audio, then receive transcription in text and speech.")
    
app.launch(show_error=True) #share=True) #uncomment share=True in google colab

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
